Bir önceki sorudaki Prensesi Iyileştir problemindeki yapay sinir ağınıza seçtiğiniz herhangi
iki farklı regülarizasyon yöntemi ekleyin

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np

# Verileri yükleme
train_data = pd.read_csv('cure_the_princess_train.csv')
test_data = pd.read_csv('cure_the_princess_test.csv')
val_data = pd.read_csv('cure_the_princess_validation.csv')

train_inputs = np.array(train_data.iloc[:, :-1])
train_targets = np.array(train_data.iloc[:, -1])
train_dataset = TensorDataset(torch.Tensor(train_inputs), torch.Tensor(train_targets))

test_inputs = np.array(test_data.iloc[:, :-1])
test_targets = np.array(test_data.iloc[:, -1])
test_dataset = TensorDataset(torch.Tensor(test_inputs), torch.Tensor(test_targets))

val_inputs = np.array(val_data.iloc[:, :-1])
val_targets = np.array(val_data.iloc[:, -1])
val_dataset = TensorDataset(torch.Tensor(val_inputs), torch.Tensor(val_targets))

batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# Model tanımlama
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.dropout(out)
        out = self.relu(self.fc2(out))
        out = self.dropout(out)
        out = self.sigmoid(self.fc3(out))
        return out

# Model eğitimi
input_dim = 13 # özellik sayısı
hidden_dim1 = 100
hidden_dim2 = 50
output_dim = 1
model = MLP(input_dim, hidden_dim1, hidden_dim2, output_dim)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, weight_decay=0.01)

num_epochs = 10
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_dataloader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(1))
        
        # L2 düzenlileştirme
        l2_reg = None
        for param in model.parameters():
            if l2_reg is None:
                l2_reg = param.norm(2)
            else:
                l2_reg = l2_reg + param.norm(2)
        
        loss += 0.001 * l2_reg
        
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print('Epoch [{}/{}], Adım [{}/{}], Kayıp: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, len(train_dataloader), loss.item()))
            

correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in test_dataloader:
        outputs = model(inputs)
        predicted = torch.round(outputs)
        total += targets.size(0)
        correct += (predicted == targets.unsqueeze(1)).sum().item()

print('Modelin test verilerindeki doğruluğu: % {} '.format(100 * correct / total))

Epoch [1/10], Adım [10/79], Kayıp: 0.6427
Epoch [1/10], Adım [20/79], Kayıp: 0.7137
Epoch [1/10], Adım [30/79], Kayıp: 0.7715
Epoch [1/10], Adım [40/79], Kayıp: 0.8365
Epoch [1/10], Adım [50/79], Kayıp: 0.7104
Epoch [1/10], Adım [60/79], Kayıp: 0.6516
Epoch [1/10], Adım [70/79], Kayıp: 0.9435
Epoch [2/10], Adım [10/79], Kayıp: 0.8959
Epoch [2/10], Adım [20/79], Kayıp: 0.7243
Epoch [2/10], Adım [30/79], Kayıp: 0.7476
Epoch [2/10], Adım [40/79], Kayıp: 0.7381
Epoch [2/10], Adım [50/79], Kayıp: 0.6861
Epoch [2/10], Adım [60/79], Kayıp: 0.6590
Epoch [2/10], Adım [70/79], Kayıp: 0.6903
Epoch [3/10], Adım [10/79], Kayıp: 0.7659
Epoch [3/10], Adım [20/79], Kayıp: 0.7703
Epoch [3/10], Adım [30/79], Kayıp: 0.6281
Epoch [3/10], Adım [40/79], Kayıp: 0.4447
Epoch [3/10], Adım [50/79], Kayıp: 0.6785
Epoch [3/10], Adım [60/79], Kayıp: 0.5312
Epoch [3/10], Adım [70/79], Kayıp: 0.6354
Epoch [4/10], Adım [10/79], Kayıp: 0.5705
Epoch [4/10], Adım [20/79], Kayıp: 0.6875
Epoch [4/10], Adım [30/79], Kayıp:

Burada, dropout katmanı ve p=0.2 ile belirtilen dropout oranı eklenmiştir. Ayrıca, weight_decay=0.01 ile belirtilen L2 düzenlileştirme parametresi de optimizasyon parametrelerine eklenmiştir. L2 düzenlileştirme, tüm ağırlıkların karesinin toplamının bir çarpanı ile kayıp fonksiyonuna eklenerek uygulanır.

Dropout ve L2 düzenlileştirme yöntemlerini kullanmamın nedeni overfitting'i önlemek ve modelin genelleme performansını arttırmaktır.

Dropout, her eğitim adımında rastgele seçilen birimleri atarak ağın öğrenme kapasitesini azaltır ve overfitting'i önler. Bu nedenle, dropout katmanı ekleyerek modelin daha iyi performans göstermesini sağlayabiliriz.

L2 düzenlileştirme, ağırlıkların büyüklüklerini sınırlayarak aşırı öğrenmeyi azaltır. Bu nedenle, L2 düzenlileştirme parametresini ekleyerek modelin daha iyi genelleme performansı elde etmesini sağlayabiliriz.

In [10]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Early stopping ile en iyi modeli seçme
best_model = None
best_val_loss = float('inf')
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_dataloader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(1))
        loss.backward()
        optimizer.step()

        # Doğrulama setindeki performansı hesaplama
        val_loss = 0
        with torch.no_grad():
            for inputs, targets in val_dataloader:
                outputs = model(inputs)
                val_loss += criterion(outputs, targets.unsqueeze(1)).item()
        val_loss /= len(val_dataloader)

        # En iyi modeli kaydetme
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = MLP(input_dim, hidden_dim1, hidden_dim2, output_dim)
            best_model.load_state_dict(model.state_dict())

    # Early stopping kontrolü
    if epoch > 0 and val_loss > prev_val_loss:
        break
    prev_val_loss = val_loss

# Test setinde performansı hesaplama
test_loss = 0
test_preds = []
test_targets = []
with torch.no_grad():
    for inputs, targets in test_dataloader:
        outputs = best_model(inputs)
        test_loss += criterion(outputs, targets.unsqueeze(1)).item()
        preds = (outputs > 0.5).float()
        test_preds.extend(preds.squeeze().tolist())
        test_targets.extend(targets.tolist())
test_loss /= len(test_dataloader)

# Performans ölçütlerini hesaplama
accuracy = accuracy_score(test_targets, test_preds)
f1 = f1_score(test_targets, test_preds)
precision = precision_score(test_targets, test_preds)
recall = recall_score(test_targets, test_preds)
success_rate = (accuracy + f1 + precision + recall) / 4 * 100

print('Test kaybı: {:.4f}'.format(test_loss))
print('Doğruluk: {:.4f}'.format(accuracy))
print('F1 puanı: {:.4f}'.format(f1))
print('Hassasiyet: {:.4f}'.format(precision))
print('Geri çağırma: {:.4f}'.format(recall))
print('Başarı Yüzdesi: {:.4f}'.format(success_rate))




Test kaybı: 0.6178
Doğruluk: 0.6490
F1 puanı: 0.6530
Hassasiyet: 0.6489
Geri çağırma: 0.6572
Başarı Yüzdesi: 65.2011
